In [1]:
%%capture
%%bash

mkdir -p svhn_data

# cd into the folder
cd svhn_data

# Now download the data
train_data="http://ufldl.stanford.edu/housenumbers/train.tar.gz"
test_data="http://ufldl.stanford.edu/housenumbers/test.tar.gz"
labels="https://github.com/krasnopis/paperclub.git"

wget $train_data 
wget $test_data
git clone $labels

In [2]:
%%capture
%%bash
cd svhn_data
# Unzip the train and test data
tar -xvf train.tar.gz
tar -xvf test.tar.gz

In [1]:
import pathlib, glob
from progressbar import ProgressBar, Percentage, Bar, ETA

def loadimgs(imglist):
    _list = []
    # Use a bit of terminal commands for faster processing
    for pimage in pbar(imglist):
        w = !file {pimage} | cut -f 2 -d ',' | cut -f 1 -d 'x'
        w = int(w[0])
        h = !file {pimage} | cut -f 2 -d ',' | cut -f 2 -d 'x'
        h = int(h[0])
        if h>80 and w>40:
            _list.append(pimage)
        
    return _list

trainfldr = pathlib.Path("svhn_data/train")
testfldr = pathlib.Path("svhn_data/test")

testimgs = glob.glob(f"{testfldr}/*.png")
trainimgs = glob.glob(f"{trainfldr}/*.png")

widgets = ['Processing test dataset: ', Percentage(), ' ', Bar(), ' ', ETA()]
pbar = ProgressBar(widgets=widgets, maxval=1e4).start()    
testimgpaths = loadimgs(testimgs)

widgets = ['Processing train dataset: ', Percentage(), ' ', Bar(), ' ', ETA()]
pbar = ProgressBar(widgets=widgets, maxval=1e4).start()    
trainimgpaths = loadimgs(trainimgs)

Processing test dataset: 100% |#################################| Time: 0:03:31
Processing train dataset: 100% |################################| Time: 0:08:59


In [2]:
# Lets sort them with numerically
import os

def sortkey(strr):
    name = os.path.basename(strr)
    return int(name.split(".")[0])

testimgpaths = sorted(testimgpaths, key=sortkey)
trainimgpaths = sorted(trainimgpaths, key=sortkey)

## Now crop out the digits from images

In [4]:
# The numbers frames are within the csv files
import numpy as np
import cv2

# To keep track of all the images 
_imgidx = 0
def crop_and_save(dinfo, img,
                  pathprefix=""):
    """
    Crop the images by processing `args` to detect where the numbers within the image are and 
    saves to the corresponding folder for the given class set by `pathprefix`.
    
    Parameters:
    -----------
    
    dinfo     : List containing the information on location of digits within the
                image, x_n, y_n, w_n, h_n, label_n, ..., with `n` denoting the nth image
    
    img       : The ndarray image containing the image from which digits are to be cropped
    """
        
    global _imgidx
    # Readng in steps of 5
    _range = len(dinfo)
    for i in range(1,_range,5):
        x,y,w,h,digit = np.asarray(dinfo[i:i+5],dtype=int)
        # It seems there are some cases where I get -1 for x_1??
        if x==-1:
            x=0
        cropped = img[y:y+h,x:x+w]
        path_cropped = f'{pathprefix}/{digit}/'
        # If the folder doesn't already exist create it
        !mkdir -p {path_cropped}
        try:
            cv2.imwrite(f"{path_cropped}{_imgidx}.png", cropped)
        except Exception as err:
            print(f"Reported err: {err}")
            print(f"dinfo: {dinfo}")
            print(f"tried to save to: {path_cropped}{_imgidx}.png")            
            
        _imgidx+=1

def process(fcsv, imgs,
            prefix=""):
    
    """
    Go through all the images, crop and save the corresponding digits
    
    Parameters:
    -----------
    
    fcsv      : A list of lines containing information on each image, where the
                the digit frames are, and what digits are there
    
    imgs      : List of SVHN images to look at
    
    prefix    : Prefix for the path where to save images
    """
    
    for img in pbar(imgs):
        # Get the index of the line within fcsv (from the {idx}.png convention)
        idx = int(os.path.basename(img).split('.png')[0])
        dinfo = fcsv[idx].split(',')
        _img = cv2.imread(img)
        crop_and_save(dinfo, _img, pathprefix=f"svhn_data/{prefix}/prepared")
    
testcsv = pathlib.Path("svhn_data/paperclub/01-SVHN/test_annotations.csv")
traincsv = pathlib.Path("svhn_data/paperclub/01-SVHN/train_annotations.csv")

f_testcsv = np.loadtxt(testcsv, dtype=object)
f_traincsv = np.loadtxt(traincsv, dtype=object)

widgets = ['Processing test dataset: ', Percentage(), ' ', Bar(), ' ', ETA()]
pbar = ProgressBar(widgets=widgets, maxval=1e4).start()    
process(f_testcsv, testimgpaths, prefix="test")

widgets = ['Processing train dataset: ', Percentage(), ' ', Bar(), ' ', ETA()]
pbar = ProgressBar(widgets=widgets, maxval=1e4).start()    
process(f_traincsv, trainimgpaths, prefix="train")

Reported err: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

dinfo: ['test/12434.png', '-3', '181', '33', '60', '8', '23', '177', '40', '60', '7']
tried to save to: svhn_data/test/prepared/8/7001.png


Processing test dataset: 100% |#################################| Time: 0:14:14
Processing train dataset: 100% |################################| Time: 0:28:27


In [5]:
# It seems 10 corresponds to zero, so lets rename those folders
!mv "svhn_data/train/prepared/10" "svhn_data/train/prepared/0"
!mv "svhn_data/test/prepared/10" "svhn_data/test/prepared/0"